In [ ]:
import requests
import json
import time
import itertools
import os
from tqdm import tqdm
import sqlite3
import datetime
import pickle
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

import Artha.twitter
import config as c
from Artha.twitter import TwitterAPI, TSQLite

checkra = TwitterAPI(username = "checkra_", bearer_token = c.c_bearer, key = c.c_key, secret = c.c_secret, token = c.c_token, token_secret = c.c_token_secret)
print(checkra.id,"done")

In [ ]:
# all_follows = checkra.get_following1("checkra_")
# with open("full_follow_list.txt", "wb") as fp: # total backup
#     pickle.dump(all_follows, fp)
# print(len(all_follows), all_follows[0])
# with open("people_left.txt", "rb") as fp:   # load progress
#     all_follows = pickle.load(fp)
# len(all_follows)
# with open("people_left.txt", "wb") as fp:  #save progress
    # pickle.dump(all_follows, fp)

In [ ]:
# people not included in databases
# all_follows.remove("259771124")
# all_follows.remove("745273")
# all_follows.remove("1354228711466426369")

In [ ]:
# process to take list of users, and get everyone that they follow
# bugs include breaking for people who follow 0 people
index = 0
# all_follows = checkra.get_following1("checkra_")
while len(all_follows)>0:
    cur_id = all_follows[0]
    cur_user = checkra.user_lookup(cur_id)["username"]

    if cur_user not in [db[1:-3] for db in os.listdir("../data/users")]:
        print("try 1", index, cur_user)
        user_follows = checkra.get_following1(cur_user)
        if isinstance(user_follows, int):  #check if too many requests
            print("try 2")
            user_follows = checkra.get_following2(cur_user)
            if isinstance(user_follows, int):  #check if too many requests
                print("Too many requests")
                time.sleep(420)
                continue
        follows_data = checkra.multiple_user_lookup(user_follows, ['id', 'name', 'username'])
        if isinstance(follows_data, int):
            print("too many req for multi_user")
            time.sleep(30)
            continue
        else:
            print("got follows for", cur_user)

        try:
            conn = sqlite3.connect("../data/users/u" + cur_user +  ".db")
            TSQLite.premaid_follow_table(conn, follows_data)
            print("db & removed", cur_user)
            all_follows.remove(cur_id)
            index += 1
        except Exception as e:
            print(e)
            break
        time.sleep(5)
        # index+=1

    else:  #if user's database already exists
        # index += 1
        print("exists, removed", cur_user)
        all_follows.remove(cur_id)


In [ ]:
rates = checkra.rate_limits()["resources"]
for key in rates["friends"].keys():
    print("Remaining:", rates["friends"][key]["remaining"], "Time:", datetime.datetime.fromtimestamp(rates["friends"][key]["reset"]), key)